# Logistic regression

let Pr(C1|x) = σ(wT x+w0) and Pr(C2|x) = 1−σ(wT x+w0). Learn the
parameters w and w0 by conditional likelihood maximization. More specifically use Newton’s algorithm
derived in class to optimize the parameters. 10 iterations of Newton’s algorithm should be sufficient for
convergence. Add a penalty of 0.5λ||w||2^2
to regularize the weights. Find the optimal hyperparameter λ
by 10-fold cross-validation.

## TODO
- Draw a graph that shows the cross-validation accuracy of logistic regression as λ varies. Report the best λ.
- Report the accuracy of logistic regression (with the best λ for regularization) on
the test set. Measure the accuracy by counting the average number of correctly labeled images. An image
is correctly labeled when the probability of the correct label is greater than 0.5.
- Print also the parameters w, w0 found for logistic regression.
- Briefly discuss the results:
    - Mixture of Gaussians and logistic regression both find a linear separator, but they use different parameterizations and different objectives. Compare the number of parameters in each model and the
amount of computation needed to find a solution with each model. Compare the results for each
model.
    - Mixture of Gaussians and logistic regression find a linear separator where as k-Nearest Neighbours
(in assignment 1) finds a non-linear separator. Compare the expressivity of the separators. Discuss
under what circumstances each type of separator is expected to perform best. What could explain
the results obtained with KNN in comparison to the results obtained with mixtures of Gaussians and
linear regression?

In [1]:
! git clone https://github.com/abnercorrea/machine-learning.git

Cloning into 'machine-learning'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 98 (delta 37), reused 84 (delta 23), pack-reused 0
Unpacking objects: 100% (98/98), done.


In [2]:
import sys
sys.path += ['/content/machine-learning/src']

In [3]:
import numpy as np
import sklearn as skl

from abnercorrea.numpy.util.data_prep import read_train_data, read_test_data, norm, prepend_col, to_binary_classes
from abnercorrea.numpy.util.data_vis import plot_alpha_scores

from abnercorrea.numpy.util.stat import logistic_sigmoid
from abnercorrea.numpy.util.data_prep import split_train_validation

# from abnercorrea.numpy.linear.logistic_regression import LogisticRegressionClassifier


# Data

In [6]:
# read data
xtrp, ytrp = read_train_data(num_partitions=10)
xtr, ytr = np.concatenate(xtrp), np.concatenate(ytrp)
xte, yte = read_test_data()

# predictors are all standardized to have mean zero and unit norm.
xtr, xte = norm(xtr - xtr.mean()), norm(xte - xte.mean())

# yi will be used as a scalar
ytr, yte = ytr[:, 0], yte[:, 0]

# X bar has 1 for the first dimension of X to accommodate for w0
xtr_, xte_ = prepend_col(xtr, 1), prepend_col(xte, 1)

# y denote the vector of yi values (yi = 1 for class 1 and yi = 0 for class 2)
ytrb, classes = to_binary_classes(ytr)
yteb, _ = to_binary_classes(yte)

In [7]:
xtr_.shape, xte_.shape, ytrb.shape, yteb.shape

((1000, 65), (110, 65), (1000,), (110,))

# PCA analysis was done in the Mixture of Gaussians notebook

# Logistic Regression
- HTF: 4.4, 5.6
- predictors are all standardized to have mean zero and unit norm
- N the number of data points
- p the number of features
- y denote the vector of yi values (yi = 1 for class 1 and yi = 0 for class 2)
- X the N × (p + 1) matrix of xi values
- p_xw the vector of fitted probabilities with ith element p(xi;βold)
- R a N×N diagonal matrix of weights with ith diagonal element p(xi;βold)(1− p(xi; βold))


























































































































































































































































































- X format: each row is an array xn

In [8]:
import logging

logger = logging.getLogger(__name__)


class LogisticRegressionClassifier:
    def __init__(self, algorithm='Newton-Raphson', max_iter=10, tol=1e-14):
        if algorithm == 'Newton-Raphson':
            self.optimizer = self.newton_raphson
        elif algorithm == 'gradient-descent':
            self.optimizer = self.gradient_descent
        else:
            raise ValueError(f'Algorithm {algorithm} not supported.')

        self.tol = tol
        self.max_iter = max_iter

    def fit(self, X_, y, alphas=None, folds=10):
        """
        Fits best w using optimizer provided and determines best alpha using cross validation.

        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        # yb denote the vector of yi values (yi = 1 for class 1 and yi = 0 for class 2)
        yb, self.classes = to_binary_classes(y)

        alpha_scores = np.zeros(len(alphas))

        for fold in range(folds):
            xtr, ytr, xvl, yvl = split_train_validation(X_, yb, fold, folds)
            for i in range(len(alphas)):
                w = self.optimizer(xtr, ytr, alphas[i], max_iter=self.max_iter, tol=self.tol)
                score = self.score(xvl, yvl, w)
                alpha_scores[i] += score

        self.alpha_scores = alpha_scores / folds
        self.alpha = alphas[alpha_scores.argmax()]
        self.w = self.optimizer(X_, yb, self.alpha, max_iter=self.max_iter, tol=self.tol)

    def newton_raphson(self, X_, y, alpha, max_iter=10, tol=1e-14, w_start=None):
        """
        This maximizes a penalized log-likelihood, l(w, alpha)

        l(w, alpha) = sum(yi @ w.T @ xi - log(1 + exp(w.T @ xi))) -.5 * alpha * w^2
        yi = 1 for class 1 and yi = 0 for class 2

        The penalty used is: -.5 * alpha * w^2 (L2)

        We typically do not penalize the intercept term, and standardize the predictors for the penalty to be meaningful.

        From HTF:

        The Newton–Raphson algorithm uses the first-derivative or Gradient and the second-derivative or Hessian matrix.

        Staring with w_old, the Newton step is:

            w_new = w_old - inverse(hessian(log_likelihood(w))) @ gradient(log_likelihood(w))

        It seems that w = 0 is a good starting value for the iterative procedure, although convergence is never guaranteed.
        Typically the algorithm does converge, since the log-likelihood is concave, but overshooting can occur.
        In the rare cases that the log-likelihood decreases, step size halving will guarantee convergence.

        This algorithm is referred to as iteratively reweighted least squares or IRLS.

        TODO: implement step-size halving in case of not converging

        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        w = w_start or np.zeros(X_.shape[1])
        iter = 0
        converged = False
        # step_size = 1

        while not converged and iter < max_iter:
            # Newton step
            gradient, p = self.gradient(X_, y, w, alpha)
            hessian = self.hessian(X_, p, alpha)
            hessian_inv = np.linalg.inv(hessian)
            # w_new = w_old - inverse(hessian(log_likelihood(w))) @ gradient(log_likelihood(w))
            w -= hessian_inv @ gradient
            # checks convergence
            converged = np.all(gradient < tol)
            iter += 1

        logger.info(f'Alpha: {alpha} - Iterations: {iter} - Converged: {converged}')

        return w

    def gradient_descent(self, X_, y, w_start, alpha, max_iter):
        """
        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        # TODO: hehe
        pass

    def score(self, X_, y, w=None, y_binary=False):
        """
        Score used is the accuracy of the model. (true positive + true negative rate)

        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        predictions = self.predict(X_, w)
        tp_tn_count = np.sum(y == predictions)
        accuracy = tp_tn_count / y.shape[0]
        return accuracy

    def predict_proba(self, X_, w=None):
        """
        Returns probability of class 0

        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        w = self.w if w is None else w
        xw = X_ @ w
        return logistic_sigmoid(xw)

    def predict(self, X_, w=None):
        """
        Prediction is a dot product... X_ @ w

        :param X_: Predictors are all standardized to have mean zero and unit norm. Predictors must contain an appened column of 1s to accommodate for w0.
        """
        w = self.w if w is None else w
        xw = X_ @ w
        predictions = np.zeros_like(xw, dtype=np.int8)
        predictions[xw >= 0] = 1
        return predictions

    def gradient(self, X_, y, w, alpha=0):
        """
        First-derivative or Gradient of log-likelihood (w)

        y denote the vector of yi values (yi = 1 for class 1 and yi = 0 for class 2)
        p vector of fitted probabilities p(xi; w)

        Gradient = X.T * (y - p) - alpha * w
        """
        xw = X_ @ w
        # vector of fitted probabilities p(xi; w)
        p = logistic_sigmoid(xw)
        # gradient vector
        gradient = X_.T @ (y - p) - alpha * w
        return gradient, p

    def hessian(self, X_, p, alpha=0):
        """
        Second-derivative or Hessian matrix of log-likelihood(w).

        Hessian = X.T @ R @ X + alpha * I
        """
        # n: # of data points
        # f: # of features + 1
        n, f = X_.shape
        # R is a N×N diagonal matrix of weights with ith diagonal element sigmoid(xi; w)(1 − sigmoid(xi; w))
        # The derivative of the sigmoid is sigmoid * (1 - sigmoid)
        R = np.zeros([n, n])
        np.fill_diagonal(R, p * (1 - p))
        # X_.T @ R @ X_ shape is (f, f)
        H = -X_.T @ R @ X_ - alpha * np.identity(f)
        return H


# Testing

In [9]:
lrc = LogisticRegressionClassifier(tol=1e-6)

In [10]:
alphas = np.arange(0, 20, 1)

In [12]:
%timeit lrc.fit(xtr_, ytr, alphas=alphas)

1 loop, best of 5: 4.67 s per loop


In [19]:
%timeit lrc.fit(xtr_, ytr, alphas=np.array([1]))

1 loop, best of 5: 353 ms per loop


In [13]:
plot_alpha_scores(alphas, lrc.alpha_scores)                                                                                                                                                                                                                                                         while                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [14]:
lrc.score(xte_, yteb)

0.9090909090909091

In [15]:
lrc.predict_proba(xte_), yte

(array([0.1082648 , 0.06054096, 0.5913902 , 0.06331777, 0.15059279,
        0.44515334, 0.90891174, 0.89695178, 0.93672377, 0.06562928,
        0.52674758, 0.00805518, 0.61544615, 0.24889922, 0.13934653,
        0.09331497, 0.10788664, 0.91536926, 0.89616293, 0.19206788,
        0.1714575 , 0.15241934, 0.86452804, 0.10124173, 0.48193289,
        0.31092134, 0.93900133, 0.9220255 , 0.73984795, 0.41429229,
        0.26368699, 0.65141384, 0.08352205, 0.67039227, 0.08296025,
        0.03331867, 0.73363472, 0.05103853, 0.49635781, 0.98890182,
        0.03049984, 0.5676605 , 0.91905741, 0.17436481, 0.07005026,
        0.76963967, 0.2154854 , 0.55858888, 0.06634599, 0.89633215,
        0.48406254, 0.00926959, 0.47770605, 0.57905203, 0.31815713,
        0.35217281, 0.95525607, 0.61309419, 0.8006296 , 0.50414201,
        0.53501233, 0.20793392, 0.03972402, 0.02601745, 0.26525062,
        0.06591946, 0.82025277, 0.93247321, 0.87355866, 0.14064811,
        0.9752388 , 0.12586524, 0.06018296, 0.70

In [ ]:
%timeit lrc.predict(xte_)

The slowest run took 150.29 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 15.9 µs per loop


In [ ]:
%timeit lrc.predict_proba(xte_)

The slowest run took 19.85 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 9.22 µs per loop


In [ ]:
%timeit lrc.fit(xtr_, ytr, alphas=alphas)

1 loop, best of 5: 6.71 s per loop


# scikit-learn comparison

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Setting to make model as equivalent as possible to my implementation
lrc_skl = LogisticRegression(penalty='l2', solver='newton-cg', tol=1e-6, max_iter=10, fit_intercept=True)

In [ ]:
%time lrc_skl.fit(xtr, ytr)

CPU times: user 24.8 ms, sys: 23 ms, total: 47.8 ms
Wall time: 31.2 ms


/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning:

newton-cg failed to converge. Increase the number of iterations.



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=1e-06, verbose=0,
                   warm_start=False)

# Very similar coeficients (w), equal for 1e-3 tolerance, same accuracy

- Coefs in my models have flipped sign compared to skl...
- TODO: how does skl optimizes alpha?

In [ ]:
lrc_skl.coef_ + lrc.w[1:]

array([[-5.96892162e-05, -7.22445010e-05,  2.37276830e-05,
         7.52254701e-05,  1.37193665e-04,  3.40146647e-05,
         1.75589273e-05, -7.59301192e-05, -4.24730564e-05,
        -4.45574659e-05,  9.62925362e-05,  7.95730166e-05,
         3.35182448e-05, -1.46860252e-05, -3.50130287e-05,
        -9.38100355e-05, -7.02416387e-05,  1.43695994e-05,
         1.00604809e-04,  3.06555974e-05, -6.39368157e-05,
        -5.88554071e-05, -5.97065376e-05, -1.00449155e-04,
        -8.43818714e-05, -2.97240517e-05,  6.64563749e-05,
         4.47784973e-05, -4.95605688e-06, -6.70664165e-06,
        -5.55099431e-05, -6.54777490e-05, -4.66872483e-05,
        -2.53863663e-05,  6.85898338e-05,  1.03248116e-04,
         2.32558056e-05,  4.68325666e-05, -4.47609153e-05,
        -1.40341117e-04, -5.96603680e-05, -3.08420855e-05,
         1.19554972e-05,  1.83023726e-05,  4.31478222e-06,
         7.20985190e-05,  3.02286307e-05, -9.44968763e-05,
        -5.57970657e-05, -3.25909467e-05,  4.47989039e-0

In [ ]:
lrc_skl.intercept_, -lrc.w[0]

(array([-0.01148148]), -0.011301769685331772)

In [ ]:
lrc_skl.score(xte, yte)

0.9090909090909091